In [ ]:
#Prophet

In [1]:
!pip install prophet

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import pandas as pd
import os
from prophet import Prophet
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "prophet_output_models"
output_csv = "prophet_output_csv"
output_metrics_csv = "prophet_metrics.csv"
output_graphs = "prophet_output_graphs"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Prepare metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Function to robustly parse dates
# -----------------------------
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

# -----------------------------
# Function to calculate MAPE
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # Convert date column safely
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')

        # Handle missing values
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled {df['Average Price'].isna().sum()} missing values with mean: {mean_value:.2f}")

        df['Average Price'] = df['Average Price'].round(2)

        # Prepare Prophet dataframe
        prophet_df = df[['Date', 'Average Price']].rename(columns={'Date': 'ds', 'Average Price': 'y'})

        # -----------------------------
        # SKIP FILES WITH < 2 VALID ROWS
        # -----------------------------
        valid_rows = prophet_df[prophet_df['y'].notna()]
        if len(valid_rows) < 2:
            print(f"❌ Skipping {file} — not enough valid rows for Prophet model.\n")
            continue

        # Build and fit Prophet model
        model = Prophet(daily_seasonality=True)
        model.fit(prophet_df)

        # Predict
        future = model.make_future_dataframe(periods=0)
        forecast = model.predict(future)

        df['Predicted'] = forecast['yhat']

        # Metrics handling
        metrics_df = df.dropna(subset=['Average Price', 'Predicted'])
        y_true = metrics_df['Average Price'].values
        y_pred = metrics_df['Predicted'].values

        df['Predicted'] = df['Predicted'].round(2)
        df.rename(columns={'Average Price': 'Actual'}, inplace=True)

        # -----------------------------
        # Calculate metrics
        # -----------------------------
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
        accuracy = round(100 - mape, 2)

        print(f"Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_prophet_model.pkl"))
        joblib.dump(model, model_file)

        # Save updated CSV
        save_df = df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_prophet_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

        # Add moving averages
        df['MA7'] = df['Actual'].rolling(7).mean()
        df['MA30'] = df['Actual'].rolling(30).mean()

        # Plot
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Actual'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red')
        plt.plot(df['Date'], df['MA7'], label='MA7', color='green', linestyle='--')
        plt.plot(df['Date'], df['MA30'], label='MA30', color='orange', linestyle='--')
        plt.title(f"{file} - Actual vs Predicted with Moving Averages")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.grid(True)
        graph_path = os.path.join(output_graphs, file.replace(".csv", "_prophet_graph.png"))
        plt.savefig(graph_path, bbox_inches='tight')
        plt.close()

# -----------------------------
# SAVE ALL METRICS
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"✅ Multiple CSVs processed! Metrics saved to {output_metrics_csv}. Models, updated CSVs, and graphs saved in respective folders.")

========== Processing: capsicum_Bangalore_daily.csv ==========


13:38:31 - cmdstanpy - INFO - Chain [1] start processing
13:38:40 - cmdstanpy - INFO - Chain [1] done processing


Metrics for capsicum_Bangalore_daily.csv:
  MAE        : 688.81
  RMSE       : 974.68
  R²         : 0.4256
  MAPE(%)    : 26.45
  Accuracy(%) : 73.55

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\capsicum_Bangalore_daily_prophet_updated.csv
========== Processing: capsicum_Belgaum_daily.csv ==========


13:38:44 - cmdstanpy - INFO - Chain [1] start processing
13:38:44 - cmdstanpy - INFO - Chain [1] done processing


Metrics for capsicum_Belgaum_daily.csv:
  MAE        : 442.34
  RMSE       : 604.33
  R²         : 0.4192
  MAPE(%)    : 15.4
  Accuracy(%) : 84.6

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\capsicum_Belgaum_daily_prophet_updated.csv
========== Processing: capsicum_Bellary_daily.csv ==========


...

Metrics for capsicum_Mysore_daily.csv:
  MAE        : 1097.13
  RMSE       : 1311.69
  R²         : -0.9465
  MAPE(%)    : 54.53
  Accuracy(%) : 45.47

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\capsicum_Mysore_daily_prophet_updated.csv
========== Processing: capsicum_Shimoga_daily.csv ==========


13:40:32 - cmdstanpy - INFO - Chain [1] start processing
13:40:32 - cmdstanpy - INFO - Chain [1] done processing


Metrics for capsicum_Shimoga_daily.csv:
  MAE        : 0.01
  RMSE       : 0.02
  R²         : 1.0
  MAPE(%)    : 0.0
  Accuracy(%) : 100.0

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\capsicum_Shimoga_daily_prophet_updated.csv
========== Processing: capsicum_Udupi_daily.csv ==========


13:40:34 - cmdstanpy - INFO - Chain [1] start processing
13:40:41 - cmdstanpy - INFO - Chain [1] done processing


Metrics for capsicum_Udupi_daily.csv:
  MAE        : 496.79
  RMSE       : 736.9
  R²         : 0.757
  MAPE(%)    : 14.62
  Accuracy(%) : 85.38

Saved updated CSV with Date, Actual & Predicted: prophet_output_csv\capsicum_Udupi_daily_prophet_updated.csv
✅ Multiple CSVs processed! Metrics saved to prophet_metrics.csv. Models, updated CSVs, and graphs saved in respective folders.


In [ ]:
#ARIMA

In [5]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "arima_output_models"
output_csv = "arima_output_csv"
output_metrics_csv = "arima_metrics.csv"
output_graphs = "arima_output_graphs"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Robust date parsing
# -----------------------------
def parse_dates_safe(date_series):
    try:
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        return pd.to_datetime(date_series, dayfirst=False)

# -----------------------------
# MAPE
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# -----------------------------
# Process CSVs
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        df = pd.read_csv(file_path)

        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')

        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled missing values with mean: {mean_value:.2f}")

        df['Average Price'] = df['Average Price'].round(2)

        # --------------- ARIMA FIT + FAILURE HANDLING ----------------
        order = (5, 1, 0)
        model = ARIMA(df['Average Price'], order=order)

        try:
            model_fit = model.fit()
        except:
            print(f"❌ Skipping {file} — ARIMA failed to fit on this dataset.\n")
            continue
        # -------------------------------------------------------------

        df['Predicted'] = model_fit.predict(start=0, end=len(df)-1)
        df['Predicted'] = df['Predicted'].round(2)

        df.rename(columns={'Average Price': 'Actual'}, inplace=True)

        y_true = df['Actual'].values
        y_pred = df['Predicted'].values

        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
        accuracy = round(100 - mape, 2)

        print(f"Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_arima_model.pkl"))
        joblib.dump(model_fit, model_file)

        # Save updated CSV
        save_df = df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_arima_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"Saved updated CSV: {updated_csv_path}")

        # Moving averages
        df['MA7'] = df['Actual'].rolling(7).mean()
        df['MA30'] = df['Actual'].rolling(30).mean()

        # Graph
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Actual'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red')
        plt.plot(df['Date'], df['MA7'], label='MA7', color='green', linestyle='--')
        plt.plot(df['Date'], df['MA30'], label='MA30', color='orange', linestyle='--')
        plt.title(f"{file} - Actual vs Predicted (ARIMA)")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.grid(True)
        graph_path = os.path.join(output_graphs, file.replace(".csv", "_arima_graph.png"))
        plt.savefig(graph_path, bbox_inches='tight')
        plt.close()

# -----------------------------
# Save metrics
# -----------------------------
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"✅ All done! Metrics saved to {output_metrics_csv}")

========== Processing: capsicum_Bangalore_daily.csv ==========
Metrics for capsicum_Bangalore_daily.csv:
  MAE        : 474.49
  RMSE       : 731.01
  R²         : 0.6583
  MAPE(%)    : 14.89
  Accuracy(%) : 85.11

Saved updated CSV: arima_output_csv\capsicum_Bangalore_daily_arima_updated.csv
========== Processing: capsicum_Belgaum_daily.csv ==========
Metrics for capsicum_Belgaum_daily.csv:
  MAE        : 82.19
  RMSE       : 249.99
  R²         : 0.9006
  MAPE(%)    : 2.87
  Accuracy(%) : 97.13

Saved updated CSV: arima_output_csv\capsicum_Belgaum_daily_arima_updated.csv
========== Processing: capsicum_Bellary_daily.csv ==========
Metrics for capsicum_Bellary_daily.csv:
  MAE        : 164.4
  RMSE       : 330.77
  R²         : 0.5417
  MAPE(%)    : 6.55
  Accuracy(%) : 93.45

Saved updated CSV: arima_output_csv\capsicum_Bellary_daily_arima_updated.csv
========== Processing: capsicum_Chikmagalur_daily.csv ==========
Metrics for capsicum_Chikmagalur_daily.csv:
  MAE        : 154.74
  R

In [ ]:
#SARIMAX

In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import joblib

warnings.filterwarnings("ignore")  # Ignore warnings

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "sarimax_output_models"
output_csv = "sarimax_output_csv"
output_graphs = "sarimax_output_graphs"
output_metrics_csv = "sarimax_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Function to robustly parse dates
# -----------------------------
def parse_dates_safe(date_series):
    try:
        # Try DD-MM-YYYY first
        return pd.to_datetime(date_series, dayfirst=True)
    except:
        # Fallback to MM-DD-YYYY
        return pd.to_datetime(date_series, dayfirst=False)

# -----------------------------
# Function to calculate MAPE
# -----------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# -----------------------------
# Process each CSV
# -----------------------------
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"========== Processing: {file} ==========")

        # Load CSV
        df = pd.read_csv(file_path)

        # -----------------------------
        # Convert 'Date' column safely
        # -----------------------------
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.sort_values('Date')
        df.set_index('Date', inplace=True)

        # Handle missing values
        if df['Average Price'].isna().sum() > 0:
            mean_value = df['Average Price'].mean()
            df['Average Price'].fillna(mean_value, inplace=True)
            print(f"Filled {df['Average Price'].isna().sum()} missing values with mean: {mean_value:.2f}")

        # Round actual values
        df['Average Price'] = df['Average Price'].round(2)

        # Add moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()

        # SARIMAX order (tune if needed)
        order = (1, 1, 1)
        seasonal_order = (1, 1, 1, 7)

        # Fit SARIMAX model
        model = SARIMAX(df['Average Price'],
                        order=order,
                        seasonal_order=seasonal_order,
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        model_fit = model.fit(disp=False)

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_sarimax_model.pkl"))
        joblib.dump(model_fit, model_file)

        # Predictions
        df['Predicted'] = model_fit.predict(start=0, end=len(df)-1)
        df['Predicted'] = df['Predicted'].round(2)  # round predictions

        # Rename Average Price to Actual
        df.rename(columns={'Average Price': 'Actual'}, inplace=True)

        # Calculate metrics
        y_true = df['Actual'].values
        y_pred = df['Predicted'].values
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(mean_absolute_percentage_error(y_true, y_pred), 2)
        accuracy = round(100 - mape, 2)

        print(f"Metrics for {file}:")
        print(f"  MAE        : {mae}")
        print(f"  RMSE       : {rmse}")
        print(f"  R²         : {r2}")
        print(f"  MAPE(%)    : {mape}")
        print(f"  Accuracy(%) : {accuracy}\n")

        # Save metrics
        metrics_list.append({
            'File': file,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'MAPE(%)': mape,
            'Accuracy(%)': accuracy
        })

        # -----------------------------
        # Save updated CSV with only Date, Actual, Predicted
        # -----------------------------
        save_df = df[['Actual', 'Predicted']].copy()
        save_df['Date'] = df.index
        save_df = save_df[['Date', 'Actual', 'Predicted']]
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_sarimax_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)
        print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

        # -----------------------------
        # Plot graph
        # -----------------------------
        plt.figure(figsize=(12,6))
        plt.plot(df.index, df['Actual'], label="Actual", color="blue")
        plt.plot(df.index, df['Predicted'], label="Predicted", color="red", linestyle="dashed")
        plt.plot(df.index, df['MA_7'], label="MA 7", color="orange")
        plt.plot(df.index, df['MA_30'], label="MA 30", color="green")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"Price Prediction (SARIMAX) - {file}")
        plt.legend()
        plt.grid(True)
        graph_file = os.path.join(output_graphs, file.replace(".csv", "_sarimax_graph.png"))
        plt.savefig(graph_file, bbox_inches='tight')
        plt.close()

# Save all metrics to CSV
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv(output_metrics_csv, index=False)
print(f"✅ Multiple CSVs processed! Metrics saved to {output_metrics_csv}. Models, updated CSVs, and graphs saved in respective folders.")

========== Processing: capsicum_Bangalore_daily.csv ==========
Metrics for capsicum_Bangalore_daily.csv:
  MAE        : 489.67
  RMSE       : 746.87
  R²         : 0.6433
  MAPE(%)    : 15.4
  Accuracy(%) : 84.6

Saved updated CSV with Date, Actual & Predicted: sarimax_output_csv\capsicum_Bangalore_daily_sarimax_updated.csv
========== Processing: capsicum_Belgaum_daily.csv ==========
Metrics for capsicum_Belgaum_daily.csv:
  MAE        : 88.75
  RMSE       : 257.56
  R²         : 0.8945
  MAPE(%)    : 3.13
  Accuracy(%) : 96.87

Saved updated CSV with Date, Actual & Predicted: sarimax_output_csv\capsicum_Belgaum_daily_sarimax_updated.csv
========== Processing: capsicum_Bellary_daily.csv ==========
Metrics for capsicum_Bellary_daily.csv:
  MAE        : 205.5
  RMSE       : 403.02
  R²         : 0.3195
  MAPE(%)    : 8.36
  Accuracy(%) : 91.64

Saved updated CSV with Date, Actual & Predicted: sarimax_output_csv\capsicum_Bellary_daily_sarimax_updated.csv
========== Processing: capsicum_Ch

In [ ]:
#TAT+MHA

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import warnings

warnings.filterwarnings("ignore")

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_output_models"
output_csv = "tat_output_csv"
output_graphs = "tat_output_graphs"
output_logs = "tat_output_logs"
metrics_file = "tat_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Create dataset sequences
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back, 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Build TAT Model (MHA + Transformer)
# -----------------------------
def build_tat_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)

    # Multi-Head Self Attention
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(inputs, inputs)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed Forward Network
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    # Global pooling + output
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Safe date parser
# -----------------------------
def parse_dates_safe(date_series):
    return pd.to_datetime(date_series, dayfirst=True, errors='coerce')

# -----------------------------
# Metrics list
# -----------------------------
metrics_list = []

# -----------------------------
# Process ALL CSV Files
# -----------------------------
look_back = 30

for file in os.listdir(input_folder):
    if file.endswith(".csv"):

        print(f"🚀 Processing: {file}")
        file_path = os.path.join(input_folder, file)

        # Load and clean
        df = pd.read_csv(file_path)
        df['Date'] = parse_dates_safe(df['Date'])
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Fill and round prices
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
        df['Average Price'] = df['Average Price'].round(2)

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(7).mean()
        df['MA_30'] = df['Average Price'].rolling(30).mean()
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # -----------------------------
        # Padding with AVERAGE for short series
        # Ensure at least (look_back + 1) rows
        # -----------------------------
        if len(df) < look_back + 1:
            missing = (look_back + 1) - len(df)
            avg_price = df['Average Price'].mean()
            min_date = df['Date'].min()

            pad_df = pd.DataFrame({
                'Date': [min_date] * missing,
                'Average Price': [avg_price] * missing,
                'MA_7': [avg_price] * missing,
                'MA_30': [avg_price] * missing
            })

            df = pd.concat([pad_df, df], ignore_index=True)
            print(f"📌 Padded {missing} rows with avg={avg_price:.2f} for {file} to reach {look_back + 1} rows.")

        # Scale values
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        # Create sequences
        X, y = create_dataset(scaled_values, look_back)

        # Just in case: if still 0 samples, create 1 dummy sample
        if len(X) == 0:
            print(f"⚠️ Still not enough sequences for {file}, creating one dummy sample.")
            pad = np.repeat(scaled_values.mean(), look_back + 1).reshape(-1, 1)
            X = np.array([pad[:look_back, 0]])
            y = np.array([pad[look_back, 0]])

        # Reshape for LSTM-style input
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_model(input_shape=(look_back, 1))

        # -----------------------------
        # FIX: Small dataset → no validation split
        # -----------------------------
        if len(X) < 5:
            val_split = 0
            print(f"⚠️ Small dataset detected for {file} → training WITHOUT validation_split.")
        else:
            val_split = 0.2

        # Train
        history = model.fit(
            X, y,
            epochs=100,
            batch_size=16,
            validation_split=val_split,
            verbose=1
        )

        # Save training logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i in range(len(history.history['loss'])):
                if val_split == 0:
                    f.write(f"Epoch {i+1}: Loss={history.history['loss'][i]}\n")
                else:
                    f.write(
                        f"Epoch {i+1}: Loss={history.history['loss'][i]}, "
                        f"Val_Loss={history.history['val_loss'][i]}\n"
                    )

        # Predict
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)

        # Add predictions to df
        df['Predicted'] = [np.nan] * look_back + list(predictions_rescaled.flatten())
        df['Predicted'] = df['Predicted'].round(2)
        df['Actual'] = df['Average Price']

        # Metrics (ignore first look_back rows with NaN preds)
        y_true = df['Actual'].values[look_back:]
        y_pred = predictions_rescaled.flatten()

        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 4)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)

        metrics_list.append([file.replace(".csv", ""), mae, rmse, r2, mape, accuracy])

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_model.h5"))
        model.save(model_file)

        # Save updated CSV
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_updated.csv"))
        df[['Date', 'Actual', 'Predicted']].to_csv(updated_csv_path, index=False)

        # Save graph
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Actual'], label='Actual', color='blue')
        plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red', linestyle='dashed')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f'Price Prediction (TAT) - {file}')
        plt.legend()
        plt.grid(True)
        graph_path = os.path.join(output_graphs, file.replace(".csv", "_tat_graph.png"))
        plt.savefig(graph_path, bbox_inches='tight')
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, ACC={accuracy}%\n")

# Save metrics CSV
metrics_df = pd.DataFrame(
    metrics_list,
    columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)']
)
metrics_df.to_csv(metrics_file, index=False)

print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: capsicum_Bangalore_daily.csv
Epoch 1/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - loss: 0.0391 - mae: 0.1520 - val_loss: 0.0220 - val_mae: 0.1024
Epoch 2/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.0152 - mae: 0.0890 - val_loss: 0.0226 - val_mae: 0.1044
Epoch 3/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 0.0163 - mae: 0.0912 - val_loss: 0.0224 - val_mae: 0.1039
Epoch 4/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - loss: 0.0160 - mae: 0.0906 - val_loss: 0.0231 - val_mae: 0.1061
Epoch 5/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 0.0154 - mae: 0.0889 - val_loss: 0.0219 - val_mae: 0.1023
Epoch 6/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - loss: 0.0156 - mae: 0.0897 - val_loss: 0.0226 - val_mae: 0.1045
Epoch 7/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.0155 - mae: 0.0893 - val_loss: 0.0243 - val_mae: 0.1100
Epoch 8/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.0154 - mae: 0.0897 - val_loss: 0.0236 - val_mae: 0.1

✅ Done with capsicum_Bangalore_daily.csv | MAE=895.16, RMSE=1263.72, R2=-0.0217, MAPE=31.73%, ACC=68.27%

🚀 Processing: capsicum_Belgaum_daily.csv
Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - loss: 0.0909 - mae: 0.2454 - val_loss: 0.0907 - val_mae: 0.2915
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0691 - mae: 0.2166 - val_loss: 0.0716 - val_mae: 0.2566
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0493 - mae: 0.1745 - val_loss: 0.0558 - val_mae: 0.2237
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0479 - mae: 0.1669 - val_loss: 0.0425 - val_mae: 0.1917
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0381 - mae: 0.1510 - val_loss: 0.0328 - val_mae: 0.1644
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0362 - mae: 0.1483 - val_loss: 0.0253 - val_mae: 0.1402
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0369 - mae: 0.1437 - val_loss: 0.0204 - val_mae: 0.1221
Epoch 8/100
23/23 ━━━━━━━━

✅ Done with capsicum_Belgaum_daily.csv | MAE=627.17, RMSE=863.9, R2=-0.0056, MAPE=22.45%, ACC=77.55%

🚀 Processing: capsicum_Bellary_daily.csv
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 0.0986 - mae: 0.1853 - val_loss: 0.0572 - val_mae: 0.2063
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - loss: 0.0980 - mae: 0.1838 - val_loss: 0.0566 - val_mae: 0.2048
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - loss: 0.0975 - mae: 0.1823 - val_loss: 0.0560 - val_mae: 0.2033
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - loss: 0.0970 - mae: 0.1808 - val_loss: 0.0554 - val_mae: 0.2018
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step - loss: 0.0964 - mae: 0.1793 - val_loss: 0.0548 - val_mae: 0.2003
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - loss: 0.0959 - mae: 0.1778 - val_loss: 0.0542 - val_mae: 0.1988
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step - loss: 0.0953 - mae: 0.1763 - val_loss: 0.0536 - val_mae: 0.1973
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2

✅ Done with capsicum_Bellary_daily.csv | MAE=306.08, RMSE=502.1, R2=-0.0603, MAPE=12.65%, ACC=87.35%

🚀 Processing: capsicum_Chikmagalur_daily.csv
Epoch 1/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 15s 26ms/step - loss: 0.0989 - mae: 0.2775 - val_loss: 0.1066 - val_mae: 0.2721
Epoch 2/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0201 - mae: 0.1052 - val_loss: 0.0779 - val_mae: 0.2382
Epoch 3/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0172 - mae: 0.0986 - val_loss: 0.0776 - val_mae: 0.2377
Epoch 4/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0170 - mae: 0.0981 - val_loss: 0.0750 - val_mae: 0.2346
Epoch 5/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0177 - mae: 0.1006 - val_loss: 0.0768 - val_mae: 0.2368
Epoch 6/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0177 - mae: 0.1021 - val_loss: 0.0777 - val_mae: 0.2379
Epoch 7/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0174 - mae: 0.0996 - val_loss: 0.0771 - val_mae: 0.2372
Epoch 8/100


✅ Done with capsicum_Chikmagalur_daily.csv | MAE=447.57, RMSE=606.64, R2=-0.0528, MAPE=24.46%, ACC=75.54%

🚀 Processing: capsicum_Davangere_daily.csv
Epoch 1/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 10s 29ms/step - loss: 0.0154 - mae: 0.0956 - val_loss: 0.0482 - val_mae: 0.1805
Epoch 2/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0067 - mae: 0.0595 - val_loss: 0.0461 - val_mae: 0.1747
Epoch 3/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0065 - mae: 0.0595 - val_loss: 0.0463 - val_mae: 0.1752
Epoch 4/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0073 - mae: 0.0635 - val_loss: 0.0462 - val_mae: 0.1751
Epoch 5/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0073 - mae: 0.0629 - val_loss: 0.0477 - val_mae: 0.1793
Epoch 6/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0065 - mae: 0.0597 - val_loss: 0.0458 - val_mae: 0.1740
Epoch 7/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0075 - mae: 0.0632 - val_loss: 0.0477 - val_mae: 0.1793
Epoch 8/100
87/87 ━━━━━

...

✅ Done with capsicum_Kolar_daily.csv | MAE=1151.18, RMSE=1631.45, R2=-0.0038, MAPE=62.57%, ACC=37.43%

🚀 Processing: capsicum_Mandya_daily.csv
Epoch 1/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.0477 - mae: 0.1543 - val_loss: 0.1270 - val_mae: 0.3387
Epoch 2/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0268 - mae: 0.1180 - val_loss: 0.1222 - val_mae: 0.3315
Epoch 3/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0256 - mae: 0.1165 - val_loss: 0.1217 - val_mae: 0.3308
Epoch 4/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0264 - mae: 0.1191 - val_loss: 0.1189 - val_mae: 0.3267
Epoch 5/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0269 - mae: 0.1217 - val_loss: 0.1189 - val_mae: 0.3268
Epoch 6/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0257 - mae: 0.1171 - val_loss: 0.1203 - val_mae: 0.3287
Epoch 7/100
284/284 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0263 - mae: 0.1175 - val_loss: 0.1166 - val_mae: 0.3232
Epoch 8/100
284/

✅ Done with capsicum_Mandya_daily.csv | MAE=918.38, RMSE=1207.16, R2=-0.0998, MAPE=58.21%, ACC=41.79%

🚀 Processing: capsicum_Mysore_daily.csv
Epoch 1/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 0.0035 - mae: 0.0443 - val_loss: 0.0042 - val_mae: 0.0445
Epoch 2/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0025 - mae: 0.0386 - val_loss: 0.0043 - val_mae: 0.0457
Epoch 3/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0024 - mae: 0.0378 - val_loss: 0.0037 - val_mae: 0.0412
Epoch 4/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0025 - mae: 0.0389 - val_loss: 0.0044 - val_mae: 0.0460
Epoch 5/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0024 - mae: 0.0380 - val_loss: 0.0051 - val_mae: 0.0519
Epoch 6/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0025 - mae: 0.0385 - val_loss: 0.0042 - val_mae: 0.0443
Epoch 7/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0025 - mae: 0.0385 - val_loss: 0.0044 - val_mae: 0.0459
Epoch 8/100
510/5

✅ Done with capsicum_Mysore_daily.csv | MAE=950.66, RMSE=1280.25, R2=-0.0431, MAPE=37.81%, ACC=62.19%

🚀 Processing: capsicum_Shimoga_daily.csv
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.6588 - mae: 0.7959 - val_loss: 0.7209 - val_mae: 0.8342
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - loss: 0.6548 - mae: 0.7934 - val_loss: 0.7167 - val_mae: 0.8317
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.6508 - mae: 0.7909 - val_loss: 0.7125 - val_mae: 0.8291
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.6468 - mae: 0.7883 - val_loss: 0.7083 - val_mae: 0.8266
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.6428 - mae: 0.7858 - val_loss: 0.7041 - val_mae: 0.8241
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 0.6388 - mae: 0.7832 - val_loss: 0.7000 - val_mae: 0.8215
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - loss: 0.6348 - mae: 0.7807 - val_loss: 0.6958 - val_mae: 0.8190
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

✅ Done with capsicum_Shimoga_daily.csv | MAE=360.39, RMSE=375.08, R2=-12.0262, MAPE=20.04%, ACC=79.96%

🚀 Processing: capsicum_Udupi_daily.csv
Epoch 1/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.0379 - mae: 0.1490 - val_loss: 0.0656 - val_mae: 0.2152
Epoch 2/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0225 - mae: 0.1254 - val_loss: 0.0640 - val_mae: 0.2118
Epoch 3/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0227 - mae: 0.1263 - val_loss: 0.0641 - val_mae: 0.2120
Epoch 4/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0228 - mae: 0.1261 - val_loss: 0.0622 - val_mae: 0.2077
Epoch 5/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0220 - mae: 0.1242 - val_loss: 0.0567 - val_mae: 0.1950
Epoch 6/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0223 - mae: 0.1265 - val_loss: 0.0595 - val_mae: 0.2016
Epoch 7/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0222 - mae: 0.1246 - val_loss: 0.0695 - val_mae: 0.2236
Epoch 8/100
247/247 ━━

✅ Done with capsicum_Udupi_daily.csv | MAE=1257.8, RMSE=1551.64, R2=-0.0763, MAPE=39.24%, ACC=60.76%

📊 Metrics saved to tat_metrics.csv


In [ ]:
#TAT+MQA

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import joblib  # for .pkl model saving (best-effort)
warnings_imported = False
try:
    import warnings
    warnings.filterwarnings("ignore")
    warnings_imported = True
except:
    pass

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_mqa_output_models"
output_csv = "tat_mqa_output_csv"
output_graphs = "tat_mqa_output_graphs"
output_logs = "tat_mqa_output_logs"
metrics_file = "tat_mqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Multi-Query Attention Layer
# -----------------------------
class MultiQueryAttention(layers.Layer):
    def __init__(self, num_heads, key_dim, dropout_rate=0.1, **kwargs):
        super(MultiQueryAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = [layers.Dense(key_dim) for _ in range(num_heads)]
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        head_outputs = []

        for q_layer in self.q_dense:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attention_output = tf.matmul(attention_weights, V)
            head_outputs.append(attention_output)

        concat = tf.concat(head_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(MultiQueryAttention, self).get_config()
        config.update({
            "num_heads": self.num_heads,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-MQA Model
# -----------------------------
def build_tat_mqa_model(input_shape, d_model=64, num_heads=4, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = MultiQueryAttention(num_heads=num_heads, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Function to robustly parse dates
# -----------------------------
def parse_dates_safe(date_series):
    # keep same behavior as you used before: infer_datetime_format True fallback
    try:
        return pd.to_datetime(date_series, infer_datetime_format=True, errors='coerce')
    except:
        return pd.to_datetime(date_series, errors='coerce')

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    file_path = os.path.join(input_folder, file)
    print(f"🚀 Processing: {file}")

    # Load CSV
    df = pd.read_csv(file_path)

    # Robust date parsing
    df['Date'] = parse_dates_safe(df['Date'])
    df = df.dropna(subset=['Date'])
    df = df.sort_values('Date').reset_index(drop=True)

    # -----------------------------
    # Handle missing Average Price
    # -----------------------------
    if 'Average Price' not in df.columns:
        raise KeyError("Input CSV must contain 'Average Price' column.")
    if df['Average Price'].isna().sum() > 0:
        mean_val = df['Average Price'].mean()
        df['Average Price'].fillna(mean_val, inplace=True)
        print(f"Filled {df['Average Price'].isna().sum()} missing Average Price values with mean {mean_val:.2f}")

    # Round Actual values early
    df['Average Price'] = df['Average Price'].astype(float).round(2)

    # Moving averages (fill remaining NaNs with column mean to avoid plotting gaps)
    df['MA_7'] = df['Average Price'].rolling(window=7).mean()
    df['MA_30'] = df['Average Price'].rolling(window=30).mean()
    df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
    df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

    # Prepare data
    values = df[['Average Price']].values.astype('float32')
    if len(values) <= look_back:
        # Not enough data for the chosen look_back; skip file with a warning
        print(f"⚠️ Skipping {file} — dataset length ({len(values)}) <= look_back ({look_back})")
        continue

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_values = scaler.fit_transform(values)

    X, y = create_dataset(scaled_values, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Build and train model
    model = build_tat_mqa_model(input_shape=(look_back,1))
    model.summary()

    history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

    # Save training logs
    log_file = os.path.join(output_logs, file.replace(".csv", "_tat_mqa_training.txt"))
    with open(log_file, "w") as f:
        f.write("Training Loss per Epoch:\n")
        for i, loss in enumerate(history.history['loss']):
            val_loss = history.history['val_loss'][i] if 'val_loss' in history.history else None
            f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={val_loss}\n")

    # Predictions (rescale)
    predictions = model.predict(X)
    predictions_rescaled = scaler.inverse_transform(predictions).flatten()
    # pad with NaNs at start so predicted series aligns with original df index
    padded_preds = np.concatenate([np.full(look_back, np.nan), predictions_rescaled])
    df['Predicted'] = np.round(padded_preds, 2)

    # -----------------------------
    # Compute metrics using only the matched portion (exclude leading NaNs)
    # -----------------------------
    y_true = df['Average Price'].values[look_back:]
    y_pred = predictions_rescaled
    mae = round(mean_absolute_error(y_true, y_pred), 2)
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
    r2 = round(r2_score(y_true, y_pred), 4)
    # MAPE: avoid divide-by-zero by masking zeros
    mask = y_true != 0
    if mask.sum() == 0:
        mape = np.nan
        accuracy = np.nan
    else:
        mape = round(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100, 2)
        accuracy = round(100 - mape, 2)

    metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

    print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%")

    # -----------------------------
    # Save model: native Keras and attempt .pkl (best-effort)
    # -----------------------------
    keras_model_path = os.path.join(output_models, file.replace(".csv", "_tat_mqa_model.keras"))
    model.save(keras_model_path)
    pkl_model_path = os.path.join(output_models, file.replace(".csv", "_tat_mqa_model.pkl"))
    try:
        # joblib.dump the Keras model object (may fail on some TF versions)
        joblib.dump(model, pkl_model_path)
    except Exception as e:
        # fallback: save model architecture + weights dict to pickle (best-effort)
        try:
            model_info = {
                "config": model.get_config(),
                "weights": model.get_weights()
            }
            joblib.dump(model_info, pkl_model_path)
        except Exception as e2:
            print(f"⚠️ Could not save .pkl for {file}: {e}; {e2}")

    # -----------------------------
    # Save CSV with only Date, Actual, Predicted (Actual renamed)
    # -----------------------------
    save_df = pd.DataFrame({
        'Date': df['Date'],
        'Actual': df['Average Price'].round(2),
        'Predicted': df['Predicted']
    })
    updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_mqa_updated.csv"))
    save_df.to_csv(updated_csv_path, index=False)
    print(f"Saved updated CSV with Date, Actual & Predicted: {updated_csv_path}")

    # -----------------------------
    # Save graph with Actual, Predicted, MA7, MA30
    # -----------------------------
    plt.figure(figsize=(12,6))
    plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
    plt.plot(df['Date'], df['Predicted'], label='Predicted', color='red', linestyle='dashed')
    plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
    plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.title(f'Price Prediction (TAT-MQA) - {file}')
    plt.legend()
    plt.grid(True)
    graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_mqa_graph.png"))
    plt.savefig(graph_file, bbox_inches='tight')
    plt.close()

# -----------------------------
# Save all metrics
# -----------------------------
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: capsicum_Bangalore_daily.csv



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_query_attention         │ (None, 30, 64)            │          17,216 │ input_layer[0][0]          │
│ (MultiQueryAttention)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 30, 64)            │               0 │ multi_query_attention[0][… │
│                               │                           │                 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization           │ (None, 30, 64)            │             128 │ add[0][0]                  │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_7 (Dense)               │ (None, 30, 128)           │           8,320 │ layer_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_8 (Dense)               │ (None, 30, 1)             │             129 │ dense_7[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_1 (Add)                   │ (None, 30, 64)            │               0 │ layer_normalization[0][0], │
│                               │                           │                 │ dense_8[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_1         │ (None, 30, 64)            │             128 │ add_1[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d      │ (None, 64)                │               0 │ layer_normalization_1[0][… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_9 (Dense)               │ (None, 1)                 │              65 │ global_average_pooling1d[… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 25,986 (101.51 KB)

 Trainable params: 25,986 (101.51 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)   │ (None, 30, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_query_attention_12      │ (None, 30, 64)            │          17,216 │ input_layer_12[0][0]       │
│ (MultiQueryAttention)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_24 (Add)                  │ (None, 30, 64)            │               0 │ multi_query_attention_12[… │
│                               │                           │                 │ input_layer_12[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_24        │ (None, 30, 64)            │             128 │ add_24[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_127 (Dense)             │ (None, 30, 128)           │           8,320 │ layer_normalization_24[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_128 (Dense)             │ (None, 30, 1)             │             129 │ dense_127[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_25 (Add)                  │ (None, 30, 64)            │               0 │ layer_normalization_24[0]… │
│                               │                           │                 │ dense_128[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_25        │ (None, 30, 64)            │             128 │ add_25[0][0]               │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d_12   │ (None, 64)                │               0 │ layer_normalization_25[0]… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_129 (Dense)             │ (None, 1)                 │              65 │ global_average_pooling1d_… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 25,986 (101.51 KB)

 Trainable params: 25,986 (101.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 0.2229 - mae: 0.2700 - val_loss: 0.0230 - val_mae: 0.1110
Epoch 2/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0116 - mae: 0.0847 - val_loss: 0.0177 - val_mae: 0.0987
Epoch 3/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0090 - mae: 0.0728 - val_loss: 0.0156 - val_mae: 0.0944
Epoch 4/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0069 - mae: 0.0636 - val_loss: 0.0157 - val_mae: 0.0941
Epoch 5/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0061 - mae: 0.0590 - val_loss: 0.0153 - val_mae: 0.0927
Epoch 6/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0050 - mae: 0.0537 - val_loss: 0.0154 - val_mae: 0.0919
Epoch 7/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0046 - mae: 0.0508 - val_loss: 0.0158 - val_mae: 0.0948
Epoch 8/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0040 - mae: 0.0472 - val_loss: 0.0155 - val_mae: 0.0902
Epoch 9/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9

In [ ]:
#TAT+GQA

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import joblib  # For saving models as .pkl

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_gqa_output_models"
output_csv = "tat_gqa_output_csv"
output_graphs = "tat_gqa_output_graphs"
output_logs = "tat_gqa_output_logs"
metrics_file = "tat_gqa_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Grouped Query Attention Layer
# -----------------------------
class GroupedQueryAttention(layers.Layer):
    def __init__(self, num_groups=2, key_dim=64, dropout_rate=0.1, **kwargs):
        super(GroupedQueryAttention, self).__init__(**kwargs)
        self.num_groups = num_groups
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.q_dense_groups = [layers.Dense(key_dim) for _ in range(num_groups)]
        self.dropout = layers.Dropout(dropout_rate)
        self.output_dense = layers.Dense(key_dim)

    def call(self, x):
        K = self.k_dense(x)
        V = self.v_dense(x)
        group_outputs = []
        for q_layer in self.q_dense_groups:
            Q = q_layer(x)
            scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(self.key_dim, tf.float32))
            attention_weights = tf.nn.softmax(scores, axis=-1)
            attn_output = tf.matmul(attention_weights, V)
            group_outputs.append(attn_output)
        concat = tf.concat(group_outputs, axis=-1)
        output = self.output_dense(concat)
        output = self.dropout(output)
        return output

    def get_config(self):
        config = super(GroupedQueryAttention, self).get_config()
        config.update({
            "num_groups": self.num_groups,
            "key_dim": self.key_dim,
            "dropout_rate": self.dropout_rate
        })
        return config

# -----------------------------
# TAT-GQA Model
# -----------------------------
def build_tat_gqa_model(input_shape, d_model=64, num_groups=2, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    x = GroupedQueryAttention(num_groups=num_groups, key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x + ff)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        # Load CSV
        df = pd.read_csv(file_path)

        # Date parsing
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Fill NaNs
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)
        X, y = create_dataset(scaled_values, look_back)

        # -------------------------
        # FIX 1: Prevent IndexError
        # -------------------------
        if X.size == 0:
            print(f"❌ Skipped {file}: Not enough rows (need > {look_back})")
            continue

        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_gqa_model(input_shape=(look_back,1), num_groups=2)

        # Train model
        history = model.fit(X, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

        # Save training logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_gqa_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy = round(100 - mape, 2)

        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_gqa_model.pkl"))
        joblib.dump(model, model_file)

        # Save updated CSV with Actual + Predicted
        save_df = df[['Date','Average Price','Predicted']].rename(columns={'Average Price':'Actual'})
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_gqa_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)

        # -------------------------
        # FIX 2: Prevent KeyError
        # -------------------------
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-GQA)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'Price Prediction (TAT-GQA) - {file}')
        plt.legend()
        plt.grid(True)

        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_gqa_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District','MAE','RMSE','R2','MAPE(%)','Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: capsicum_Bangalore_daily.csv


C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2878872630.py:115: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2878872630.py:120: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

Epoch 1/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.1247 - mae: 0.1988 - val_loss: 0.0125 - val_mae: 0.0794
Epoch 2/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0106 - mae: 0.0749 - val_loss: 0.0118 - val_mae: 0.0792
Epoch 3/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0079 - mae: 0.0642 - val_loss: 0.0118 - val_mae: 0.0832
Epoch 4/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0071 - mae: 0.0610 - val_loss: 0.0159 - val_mae: 0.1026
Epoch 5/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0064 - mae: 0.0568 - val_loss: 0.0117 - val_mae: 0.0813
Epoch 6/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0067 - mae: 0.0580 - val_loss: 0.0128 - val_mae: 0.0890
Epoch 7/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0066 - mae: 0.0566 - val_loss: 0.0118 - val_mae: 0.0826
Epoch 8/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0066 - mae: 0.0574 - val_loss: 0.0120 - val_mae: 0.0845
Epoch 9/100
463/463 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/

C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2878872630.py:115: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2878872630.py:120: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

Epoch 1/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.8333 - mae: 0.6508 - val_loss: 0.0039 - val_mae: 0.0449
Epoch 2/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0171 - mae: 0.0992 - val_loss: 0.0041 - val_mae: 0.0584
Epoch 3/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0154 - mae: 0.0932 - val_loss: 0.0027 - val_mae: 0.0397
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0152 - mae: 0.0926 - val_loss: 0.0034 - val_mae: 0.0382
Epoch 5/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0149 - mae: 0.0902 - val_loss: 0.0026 - val_mae: 0.0428
Epoch 6/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0146 - mae: 0.0883 - val_loss: 0.0094 - val_mae: 0.0885
Epoch 7/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0148 - mae: 0.0925 - val_loss: 0.0035 - val_mae: 0.0395
Epoch 8/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0152 - mae: 0.0885 - val_loss: 0.0027 - val_mae: 0.0394
Epoch 9/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.01

...

Epoch 1/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 0.0702 - mae: 0.1456 - val_loss: 0.0030 - val_mae: 0.0439
Epoch 2/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0035 - mae: 0.0460 - val_loss: 0.0030 - val_mae: 0.0443
Epoch 3/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0023 - mae: 0.0367 - val_loss: 0.0020 - val_mae: 0.0317
Epoch 4/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0017 - mae: 0.0316 - val_loss: 0.0020 - val_mae: 0.0321
Epoch 5/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0015 - mae: 0.0290 - val_loss: 0.0020 - val_mae: 0.0315
Epoch 6/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0014 - mae: 0.0279 - val_loss: 0.0020 - val_mae: 0.0327
Epoch 7/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0014 - mae: 0.0271 - val_loss: 0.0021 - val_mae: 0.0327
Epoch 8/100
510/510 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0015 - mae: 0.0282 - val_loss: 0.0022 - val_mae: 0.0325
Epoch 9/100
510/510 ━━━━━━━━━━━━━━━━━━━

C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2878872630.py:115: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2878872630.py:120: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 181ms/step - loss: 0.5979 - mae: 0.6911 - val_loss: 0.0607 - val_mae: 0.2417
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.1285 - mae: 0.2904 - val_loss: 0.3662 - val_mae: 0.6037
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0994 - mae: 0.2596 - val_loss: 0.0104 - val_mae: 0.0934
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0889 - mae: 0.2594 - val_loss: 0.0988 - val_mae: 0.3122
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0556 - mae: 0.1947 - val_loss: 0.0183 - val_mae: 0.1305
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0218 - mae: 0.1120 - val_loss: 0.0061 - val_mae: 0.0700
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0264 - mae: 0.1341 - val_loss: 0.0250 - val_mae: 0.1546
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0308 - mae: 0.1382 - val_loss: 0.0022 - val_mae: 0.0378
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0261 - mae:

C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2878872630.py:115: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2878872630.py:120: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

Epoch 1/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - loss: 0.1455 - mae: 0.2342 - val_loss: 0.0170 - val_mae: 0.0992
Epoch 2/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0118 - mae: 0.0837 - val_loss: 0.0161 - val_mae: 0.0954
Epoch 3/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0089 - mae: 0.0724 - val_loss: 0.0156 - val_mae: 0.0946
Epoch 4/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0066 - mae: 0.0627 - val_loss: 0.0158 - val_mae: 0.0938
Epoch 5/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0056 - mae: 0.0560 - val_loss: 0.0150 - val_mae: 0.0912
Epoch 6/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0046 - mae: 0.0505 - val_loss: 0.0154 - val_mae: 0.0905
Epoch 7/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0033 - mae: 0.0421 - val_loss: 0.0155 - val_mae: 0.0895
Epoch 8/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.0031 - mae: 0.0405 - val_loss: 0.0165 - val_mae: 0.0911
Epoch 9/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 

In [ ]:
#TAT+HA

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# -----------------------------
# Output directories
# -----------------------------
input_folder = "dataset"
output_models = "tat_ha_output_models"
output_csv = "tat_ha_output_csv"
output_graphs = "tat_ha_output_graphs"
output_logs = "tat_ha_output_logs"
metrics_file = "tat_ha_metrics.csv"

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])
        y.append(data[i+look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Local Attention Layer
# -----------------------------
class LocalAttention(layers.Layer):
    def __init__(self, key_dim=64, dropout_rate=0.1, **kwargs):
        super(LocalAttention, self).__init__(**kwargs)
        self.key_dim = key_dim
        self.dropout_rate = dropout_rate
        self.q_dense = layers.Dense(key_dim)
        self.k_dense = layers.Dense(key_dim)
        self.v_dense = layers.Dense(key_dim)
        self.dropout = layers.Dropout(dropout_rate)

    def call(self, x):
        Q = self.q_dense(x)
        K = self.k_dense(x)
        V = self.v_dense(x)
        scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(tf.shape(K)[-1], tf.float32))
        weights = tf.nn.softmax(scores, axis=-1)
        output = tf.matmul(weights, V)
        output = self.dropout(output)
        return output

# -----------------------------
# Hierarchical Attention Model
# -----------------------------
def build_tat_ha_model(input_shape, d_model=64, ff_dim=128, dropout_rate=0.2):
    inputs = layers.Input(shape=input_shape)
    local_attn = LocalAttention(key_dim=d_model, dropout_rate=dropout_rate)(inputs)
    local_attn = layers.LayerNormalization(epsilon=1e-6)(local_attn + inputs)
    global_attn = layers.MultiHeadAttention(num_heads=4, key_dim=d_model)(local_attn, local_attn)
    global_attn = layers.Dropout(dropout_rate)(global_attn)
    global_attn = layers.LayerNormalization(epsilon=1e-6)(global_attn + local_attn)
    ff = layers.Dense(ff_dim, activation='relu')(global_attn)
    ff = layers.Dense(input_shape[1])(ff)
    x = layers.LayerNormalization(epsilon=1e-6)(ff + global_attn)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mae'])
    return model

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(input_folder, file)
        print(f"🚀 Processing: {file}")

        df = pd.read_csv(file_path)

        # Date parsing
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Handle missing Average Price
        df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)

        # Moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean()
        df['MA_30'] = df['Average Price'].rolling(window=30).mean()
        df['MA_7'].fillna(df['MA_7'].mean(), inplace=True)
        df['MA_30'].fillna(df['MA_30'].mean(), inplace=True)

        # Prepare data
        values = df[['Average Price']].values.astype('float32')
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)
        X, y = create_dataset(scaled_values, look_back)

        # -------------------------------------
        # ❗ FIX 1 — Prevent IndexError
        # -------------------------------------
        if X.size == 0:
            print(f"❌ Skipped {file}: Not enough rows to form a sequence (need > {look_back})")
            continue

        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Build model
        model = build_tat_ha_model(input_shape=(look_back,1))

        # Train
        history = model.fit(X, y, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

        # Save logs
        log_file = os.path.join(output_logs, file.replace(".csv", "_tat_ha_training.txt"))
        with open(log_file, "w") as f:
            f.write("Training Loss per Epoch:\n")
            for i, loss in enumerate(history.history['loss']):
                f.write(f"Epoch {i+1}: Loss={loss}, Val_Loss={history.history['val_loss'][i]}\n")

        # Predictions
        predictions = model.predict(X)
        predictions_rescaled = scaler.inverse_transform(predictions)
        df['Predicted'] = [np.nan]*look_back + list(predictions_rescaled.flatten())

        # Round values
        df['Average Price'] = df['Average Price'].round(2)
        df['Predicted'] = df['Predicted'].round(2)

        # Metrics
        y_true = df['Average Price'].values[look_back:]
        y_pred = predictions_rescaled.flatten()
        mae = round(mean_absolute_error(y_true, y_pred), 2)
        rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 2)
        r2 = round(r2_score(y_true, y_pred), 2)
        mape = round(np.mean(np.abs((y_true - y_pred)/y_true))*100, 2)
        accuracy = round(100 - mape, 2)
        metrics_list.append([file.replace(".csv",""), mae, rmse, r2, mape, accuracy])

        # Save model
        model_file = os.path.join(output_models, file.replace(".csv", "_tat_ha_model.pkl"))
        joblib.dump(model, model_file)

        # Save updated CSV
        save_df = df[['Date', 'Average Price', 'Predicted']].rename(columns={'Average Price': 'Actual'})
        updated_csv_path = os.path.join(output_csv, file.replace(".csv", "_tat_ha_updated.csv"))
        save_df.to_csv(updated_csv_path, index=False)

        # -------------------------------------
        # ❗ FIX 2 — Prevent KeyError
        # -------------------------------------
        plt.figure(figsize=(12,6))
        plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df['Date'], df['Predicted'], label='Predicted (TAT-HA)', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.title(f'Price Prediction (TAT-HA) - {file}')
        plt.legend()
        plt.grid(True)

        graph_file = os.path.join(output_graphs, file.replace(".csv", "_tat_ha_graph.png"))
        plt.savefig(graph_file)
        plt.close()

        print(f"✅ Done with {file} | MAE={mae}, RMSE={rmse}, R2={r2}, MAPE={mape}%, Accuracy={accuracy}%\n")

# Save metrics
metrics_df = pd.DataFrame(metrics_list, columns=['District','MAE','RMSE','R2','MAPE(%)','Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"📊 Metrics saved to {metrics_file}")

🚀 Processing: capsicum_Bangalore_daily.csv


C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2714984772.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2714984772.py:106: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh


Epoch 1/50
463/463 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - loss: 0.0424 - mae: 0.1320 - val_loss: 0.0120 - val_mae: 0.0822
Epoch 2/50
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.0093 - mae: 0.0696 - val_loss: 0.0123 - val_mae: 0.0879
Epoch 3/50
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.0068 - mae: 0.0594 - val_loss: 0.0115 - val_mae: 0.0799
Epoch 4/50
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.0068 - mae: 0.0577 - val_loss: 0.0188 - val_mae: 0.1153
Epoch 5/50
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.0072 - mae: 0.0613 - val_loss: 0.0138 - val_mae: 0.0948
Epoch 6/50
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0067 - mae: 0.0585 - val_loss: 0.0157 - val_mae: 0.1023
Epoch 7/50
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.0066 - mae: 0.0576 - val_loss: 0.0147 - val_mae: 0.0993
Epoch 8/50
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 0.0063 - mae: 0.0567 - val_loss: 0.0153 - val_mae: 0.1015
Epoch 9/50
463/463 ━━━━━━━━━━━━━━━━━━━━ 7s 14m

C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2714984772.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2714984772.py:106: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - loss: 0.2671 - mae: 0.3698 - val_loss: 0.0050 - val_mae: 0.0643
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0168 - mae: 0.0980 - val_loss: 0.0043 - val_mae: 0.0598
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0168 - mae: 0.1006 - val_loss: 0.0062 - val_mae: 0.0717
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0166 - mae: 0.0997 - val_loss: 0.0043 - val_mae: 0.0600
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0159 - mae: 0.0904 - val_loss: 0.0032 - val_mae: 0.0506
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0124 - mae: 0.0813 - val_loss: 0.0089 - val_mae: 0.0859
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0139 - mae: 0.0886 - val_loss: 0.0034 - val_mae: 0.0388
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0131 - mae: 0.0859 - val_loss: 0.0039 - val_mae: 0.0568
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0108 - mae: 0.

...

Epoch 1/50
510/510 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - loss: 0.1369 - mae: 0.1710 - val_loss: 0.0024 - val_mae: 0.0372
Epoch 2/50
510/510 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 0.0025 - mae: 0.0392 - val_loss: 0.0021 - val_mae: 0.0324
Epoch 3/50
510/510 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - loss: 0.0020 - mae: 0.0341 - val_loss: 0.0022 - val_mae: 0.0324
Epoch 4/50
510/510 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - loss: 0.0020 - mae: 0.0337 - val_loss: 0.0021 - val_mae: 0.0324
Epoch 5/50
510/510 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 0.0016 - mae: 0.0292 - val_loss: 0.0024 - val_mae: 0.0336
Epoch 6/50
510/510 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 0.0016 - mae: 0.0295 - val_loss: 0.0067 - val_mae: 0.0734
Epoch 7/50
510/510 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - loss: 0.0015 - mae: 0.0294 - val_loss: 0.0058 - val_mae: 0.0675
Epoch 8/50
510/510 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - loss: 0.0017 - mae: 0.0305 - val_loss: 0.0023 - val_mae: 0.0360
Epoch 9/50
510/510 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms

C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2714984772.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2714984772.py:106: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 183ms/step - loss: 0.3414 - mae: 0.5094 - val_loss: 0.6273 - val_mae: 0.7909
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.2234 - mae: 0.4327 - val_loss: 0.0076 - val_mae: 0.0757
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1024 - mae: 0.2693 - val_loss: 0.0334 - val_mae: 0.1780
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0226 - mae: 0.1162 - val_loss: 0.2174 - val_mae: 0.4646
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0595 - mae: 0.2050 - val_loss: 0.0063 - val_mae: 0.0689
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0319 - mae: 0.1546 - val_loss: 0.0149 - val_mae: 0.1160
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0208 - mae: 0.1188 - val_loss: 0.1122 - val_mae: 0.3329
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0333 - mae: 0.1538 - val_loss: 0.0079 - val_mae: 0.0804
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0234 - mae: 0.1276 -

C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2714984772.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Average Price'].fillna(df['Average Price'].mean(), inplace=True)
C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\2714984772.py:106: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

Epoch 1/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - loss: 0.1110 - mae: 0.2106 - val_loss: 0.0226 - val_mae: 0.1099
Epoch 2/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0090 - mae: 0.0739 - val_loss: 0.0152 - val_mae: 0.0934
Epoch 3/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0043 - mae: 0.0474 - val_loss: 0.0137 - val_mae: 0.0851
Epoch 4/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0036 - mae: 0.0436 - val_loss: 0.0175 - val_mae: 0.1042
Epoch 5/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0034 - mae: 0.0404 - val_loss: 0.0196 - val_mae: 0.1121
Epoch 6/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0025 - mae: 0.0354 - val_loss: 0.0190 - val_mae: 0.1094
Epoch 7/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0027 - mae: 0.0366 - val_loss: 0.0151 - val_mae: 0.0903
Epoch 8/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0023 - mae: 0.0312 - val_loss: 0.0171 - val_mae: 0.0992
Epoch 9/50
247/247 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/

In [ ]:
#LSTM

In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# -----------------------------
# Paths
# -----------------------------
input_folder = "dataset"
output_models = "output_lstm_models"
output_csv = "output_lstm_csv"
output_graphs = "output_lstm_graphs"
output_logs = "output_lstm_logs"
metrics_file = os.path.join(output_csv, "lstm_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create sequences
# -----------------------------
def create_sequences(data, seq_length=5):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV
# -----------------------------
seq_length = 5
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n========== Processing: {file} ==========")

        # Load CSV with multiple date formats
        df = pd.read_csv(os.path.join(input_folder, file))
        df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Fill missing Average Price
        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        # Moving averages (for plotting only)
        df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

        # Scaling
        scaler = MinMaxScaler()
        prices_scaled = scaler.fit_transform(df['Average Price'].values.reshape(-1, 1))

        # Prepare sequences
        X, y = create_sequences(prices_scaled, seq_length)

        # Train-test split (80%-20%)
        split = int(0.8 * len(X))
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Build LSTM model
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])

        # Train model
        history = model.fit(
            X_train, y_train,
            epochs=20,
            batch_size=32,
            validation_data=(X_test, y_test),
            verbose=1
        )

        # Predictions
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Round Actual & Predicted
        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        # Save predictions in CSV (only Date, Actual, Predicted)
        df_pred = df.iloc[seq_length + split:].copy()
        df_pred = df_pred[['Date']].copy()
        df_pred['Actual'] = y_true_round.flatten()
        df_pred['Predicted'] = y_pred_round.flatten()
        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_lstm_updated.csv"), index=False)

        # Metrics calculation
        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(mean_squared_error(y_true, y_pred, squared=False), 2)
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy_val = round(100 - mape_val, 2)
        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])

        print(f"✅ Done with {file} | MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, "
              f"MAPE={mape_val}%, Accuracy={accuracy_val}%")

        # Save model as .pkl
        model_file = os.path.join(output_models, f"{district_name}_lstm_model.pkl")
        joblib.dump(model, model_file)

        # Save prediction graph (with MA7 & MA30 only for graphing)
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Average Price'], label='Actual', color='blue')
        plt.plot(df['Date'], df['MA_7'], label='MA 7', color='orange')
        plt.plot(df['Date'], df['MA_30'], label='MA 30', color='green')
        plt.plot(df_pred['Date'], df_pred['Predicted'], label='Predicted', color='red', linestyle='dashed')
        plt.xlabel('Date')
        plt.ylabel('Average Price')
        plt.title(f"LSTM Predictions for {district_name}")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_graph.png"))
        plt.close()

        # Save training loss graph
        plt.figure(figsize=(8, 4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f"Training Loss for {district_name}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_lstm_training_loss.png"))
        plt.close()

# Save metrics CSV
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print(f"\n📊 Metrics saved to {metrics_file}")
print("\nAll districts processed successfully!")


========== Processing: capsicum_Bangalore_daily.csv ==========


C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\3711824180.py:52: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
C:\Users\ravik\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0283 - mae: 0.1187 - val_loss: 0.0118 - val_mae: 0.0820
Epoch 2/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0054 - mae: 0.0464 - val_loss: 0.0117 - val_mae: 0.0831
Epoch 3/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0054 - mae: 0.0453 - val_loss: 0.0122 - val_mae: 0.0840
Epoch 4/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0059 - mae: 0.0474 - val_loss: 0.0117 - val_mae: 0.0818
Epoch 5/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0055 - mae: 0.0467 - val_loss: 0.0118 - val_mae: 0.0813
Epoch 6/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0050 - mae: 0.0443 - val_loss: 0.0116 - val_mae: 0.0829
Epoch 7/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0053 - mae: 0.0455 - val_loss: 0.0115 - val_mae: 0.0825
Epoch 8/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0051 - mae: 0.0448 - val_loss: 0.0116 - val_mae: 0.0831
Epoch 9/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - lo

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



========== Processing: capsicum_Belgaum_daily.csv ==========
Epoch 1/20


...


========== Processing: capsicum_Udupi_daily.csv ==========
Epoch 1/20


C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\3711824180.py:52: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True, errors='coerce')
C:\Users\ravik\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0157 - mae: 0.0806 - val_loss: 0.0040 - val_mae: 0.0344
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 5.7202e-04 - mae: 0.0090 - val_loss: 0.0038 - val_mae: 0.0354
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.9146e-04 - mae: 0.0085 - val_loss: 0.0035 - val_mae: 0.0320
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.0982e-04 - mae: 0.0080 - val_loss: 0.0034 - val_mae: 0.0315
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.9001e-04 - mae: 0.0078 - val_loss: 0.0036 - val_mae: 0.0370
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 4.0213e-04 - mae: 0.0109 - val_loss: 0.0030 - val_mae: 0.0287
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.3739e-04 - mae: 0.0076 - val_loss: 0.0029 - val_mae: 0.0268
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.7715e-04 - mae: 0.0089 - val_loss: 0.0027 - val_mae: 0.0271
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



📊 Metrics saved to output_lstm_csv\lstm_metrics.csv

All districts processed successfully!


In [ ]:
#GRU

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

# -----------------------------
# Output paths
# -----------------------------
input_folder = "dataset"
output_models = "gru_output_models"
output_csv = "gru_output_csv"
output_graphs = "gru_output_graphs"
output_logs = "gru_output_logs"
metrics_file = os.path.join(output_csv, "gru_metrics.csv")

os.makedirs(output_models, exist_ok=True)
os.makedirs(output_csv, exist_ok=True)
os.makedirs(output_graphs, exist_ok=True)
os.makedirs(output_logs, exist_ok=True)

# -----------------------------
# Function to create dataset
# -----------------------------
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i + look_back, 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

# -----------------------------
# Metrics storage
# -----------------------------
metrics_list = []

# -----------------------------
# Process each CSV file
# -----------------------------
look_back = 30
for file in os.listdir(input_folder):
    if file.endswith(".csv"):
        district_name = file.split(".")[0]
        print(f"\n🚀 Processing: {file}")

        # Load dataset
        df = pd.read_csv(os.path.join(input_folder, file))

        # Robust date parsing
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce', infer_datetime_format=True)
        df = df.dropna(subset=['Date'])
        df = df.sort_values('Date')

        # Handle missing values
        df['Average Price'] = df['Average Price'].fillna(df['Average Price'].mean())

        # Add moving averages
        df['MA_7'] = df['Average Price'].rolling(window=7).mean().fillna(df['Average Price'].mean())
        df['MA_30'] = df['Average Price'].rolling(window=30).mean().fillna(df['Average Price'].mean())

        # Scale data
        values = df[['Average Price']].values
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_values = scaler.fit_transform(values)

        X, y = create_dataset(scaled_values, look_back)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        # Train-test split
        split = int(len(X) * 0.8)
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Build GRU model
        model = Sequential([
            GRU(64, return_sequences=True, input_shape=(look_back, 1)),
            Dropout(0.2),
            GRU(32),
            Dropout(0.2),
            Dense(1)
        ])
        model.compile(optimizer="adam", loss="mse", metrics=['mae'])

        # Train model
        history = model.fit(X_train, y_train, epochs=20, batch_size=16,
                            validation_data=(X_test, y_test), verbose=1)

        # Save training log
        log_file = os.path.join(output_logs, f"{district_name}_gru_training.txt")
        with open(log_file, "w") as f:
            f.write("Epoch\tTrain_Loss\tVal_Loss\n")
            for i, (loss, val_loss) in enumerate(zip(history.history['loss'], history.history['val_loss'])):
                f.write(f"{i+1}\t{loss:.6f}\t{val_loss:.6f}\n")

        # Predictions
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_true = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Round predictions
        y_true_round = np.round(y_true, 2)
        y_pred_round = np.round(y_pred, 2)

        # Save predictions CSV (only Date, Actual, Predicted)
        df_pred = df.iloc[-len(y_true):].copy()
        df_pred = df_pred[['Date']].reset_index(drop=True)
        df_pred['Actual'] = y_true_round.flatten()
        df_pred['Predicted'] = y_pred_round.flatten()
        df_pred.to_csv(os.path.join(output_csv, f"{district_name}_gru_updated.csv"), index=False)

        # Metrics
        mae_val = round(mean_absolute_error(y_true, y_pred), 2)
        rmse_val = round(mean_squared_error(y_true, y_pred, squared=False), 2)
        r2_val = round(r2_score(y_true, y_pred), 2)
        mape_val = round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
        accuracy_val = round(100 - mape_val, 2)
        metrics_list.append([district_name, mae_val, rmse_val, r2_val, mape_val, accuracy_val])

        print(f"✅ Done with {file} | MAE={mae_val}, RMSE={rmse_val}, R2={r2_val}, "
              f"MAPE={mape_val}%, Accuracy={accuracy_val}%")

        # Save model as .pkl
        model_file = os.path.join(output_models, f"{district_name}_gru_model.pkl")
        joblib.dump(model, model_file)

        # Save prediction graph (with MA7 & MA30)
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['Average Price'], label="Actual", color="blue")
        plt.plot(df['Date'], df['MA_7'], label="MA_7", color="orange")
        plt.plot(df['Date'], df['MA_30'], label="MA_30", color="green")
        plt.plot(df_pred['Date'], df_pred['Predicted'], label="Predicted (GRU)", color="red", linestyle="dashed")
        plt.xlabel("Date")
        plt.ylabel("Average Price")
        plt.title(f"GRU Predictions - {district_name}")
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_graph.png"))
        plt.close()

        # Training loss graph
        plt.figure(figsize=(8, 4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Val Loss')
        plt.title(f"GRU Training Loss - {district_name}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(os.path.join(output_graphs, f"{district_name}_gru_loss.png"))
        plt.close()

# Save metrics CSV
metrics_df = pd.DataFrame(metrics_list, columns=['District', 'MAE', 'RMSE', 'R2', 'MAPE(%)', 'Accuracy(%)'])
metrics_df.to_csv(metrics_file, index=False)
print("\n📊 Metrics saved to", metrics_file)
print("\n✅ All districts processed successfully!")


🚀 Processing: capsicum_Bangalore_daily.csv
Epoch 1/20


C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\1911045302.py:54: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce', infer_datetime_format=True)
C:\Users\ravik\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


463/463 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - loss: 0.0093 - mae: 0.0650 - val_loss: 0.0124 - val_mae: 0.0812
Epoch 2/20
463/463 ━━━━━━━━━━━━━━━━━━━━ 14s 30ms/step - loss: 0.0060 - mae: 0.0519 - val_loss: 0.0106 - val_mae: 0.0800
Epoch 3/20
463/463 ━━━━━━━━━━━━━━━━━━━━ 13s 28ms/step - loss: 0.0058 - mae: 0.0506 - val_loss: 0.0100 - val_mae: 0.0757
Epoch 4/20
463/463 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - loss: 0.0050 - mae: 0.0469 - val_loss: 0.0105 - val_mae: 0.0758
Epoch 5/20
463/463 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - loss: 0.0055 - mae: 0.0475 - val_loss: 0.0100 - val_mae: 0.0758
Epoch 6/20
463/463 ━━━━━━━━━━━━━━━━━━━━ 16s 35ms/step - loss: 0.0051 - mae: 0.0470 - val_loss: 0.0101 - val_mae: 0.0751
Epoch 7/20
463/463 ━━━━━━━━━━━━━━━━━━━━ 12s 26ms/step - loss: 0.0051 - mae: 0.0472 - val_loss: 0.0107 - val_mae: 0.0763
Epoch 8/20
463/463 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - loss: 0.0052 - mae: 0.0463 - val_loss: 0.0103 - val_mae: 0.0754
Epoch 9/20
463/463 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/st

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



🚀 Processing: capsicum_Belgaum_daily.csv
Epoch 1/20


...


🚀 Processing: capsicum_Udupi_daily.csv
Epoch 1/20


C:\Users\ravik\AppData\Local\Temp\ipykernel_20680\1911045302.py:54: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce', infer_datetime_format=True)
C:\Users\ravik\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


247/247 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - loss: 0.0080 - mae: 0.0523 - val_loss: 0.0034 - val_mae: 0.0326
Epoch 2/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.0012 - mae: 0.0220 - val_loss: 0.0029 - val_mae: 0.0289
Epoch 3/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 9.3509e-04 - mae: 0.0192 - val_loss: 0.0024 - val_mae: 0.0226
Epoch 4/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 8.4787e-04 - mae: 0.0181 - val_loss: 0.0022 - val_mae: 0.0215
Epoch 5/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 6.9555e-04 - mae: 0.0166 - val_loss: 0.0022 - val_mae: 0.0288
Epoch 6/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 6.7212e-04 - mae: 0.0165 - val_loss: 0.0021 - val_mae: 0.0280
Epoch 7/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 6.2960e-04 - mae: 0.0154 - val_loss: 0.0016 - val_mae: 0.0178
Epoch 8/20
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 5.5326e-04 - mae: 0.0146 - val_loss: 0.0018 - val_mae: 0.0240
Epoch 9/20
247/247 ━━━━━━━━━━━━━━━━

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



📊 Metrics saved to gru_output_csv\gru_metrics.csv

✅ All districts processed successfully!
